# Machine Learning Project - Kickstarter Data Set
## Data Cleaning Notebook
*Contributor: Max Langer, René Ebrecht, Jens Reich*

This is the very first project where we build a machine learning model from scratch based on an unknown dataset.
The dataset includes data from Kickstarter projects from the years 2009 to 2019.

Our goal is to help our (fictional) stackholder, PPC Consultants with a model that can predict whether a Kickstarter project will be successful or not. 
PPC Consultants advises potential project creators (PPCs) with their projects to get them off the ground as successfully as possible.
Therefore, the value of our data product (the predictive model) is to show opportunities, save time, and in the end make money for both PPC consultants and PPCs.

In [34]:
# Import the organization modules
import pandas as pd
import numpy as np
# Import module to ignore warnings
import warnings
warnings.filterwarnings('ignore')
# Import the plot modules
import matplotlib.pyplot as plt
import seaborn as sns
# Import own scripts
from scripts.data_cleaning import (
    read_all_csvs, 
    create_csv, 
    get_nan_cols, 
    convert_to_datetime, 
    calculate_time_periods, 
    get_year_month_day, 
    entangle_column,
    get_char_len,
    one_hot_encode
    )

In [35]:
# Create data frame from all single CSV files
df = read_all_csvs()

In [36]:
# Take a look at the first columns
df.head()

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,21,2006 was almost 7 years ago.... Can you believ...,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po...",802,US,1387659690,"{""id"":1495925645,""name"":""Daniel"",""is_registere...",USD,$,True,...,new-final-round-album,https://www.kickstarter.com/discover/categorie...,True,False,successful,1391899046,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",802.00000,international
1,97,An adorable fantasy enamel pin series of princ...,"{""id"":54,""name"":""Mixed Media"",""slug"":""art/mixe...",2259,US,1549659768,"{""id"":1175589980,""name"":""Katherine"",""slug"":""fr...",USD,$,True,...,princess-pals-enamel-pin-series,https://www.kickstarter.com/discover/categorie...,True,False,successful,1551801611,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",2259.00000,international
2,88,Helping a community come together to set the s...,"{""id"":280,""name"":""Photobooks"",""slug"":""photogra...",29638,US,1477242384,"{""id"":1196856269,""name"":""MelissaThomas"",""is_re...",USD,$,True,...,their-life-through-their-lens-the-amish-and-me...,https://www.kickstarter.com/discover/categorie...,True,True,successful,1480607932,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",29638.00000,international
3,193,Every revolution starts from the bottom and we...,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",49158,IT,1540369920,"{""id"":1569700626,""name"":""WAO"",""slug"":""wearewao...",EUR,€,False,...,wao-the-eco-effect-shoes,https://www.kickstarter.com/discover/categorie...,True,False,successful,1544309940,1.136525,"{""web"":{""project"":""https://www.kickstarter.com...",49075.15252,international
4,20,Learn to build 10+ Applications in this comple...,"{""id"":51,""name"":""Software"",""slug"":""technology/...",549,US,1425706517,"{""id"":1870845385,""name"":""Kalpit Jain"",""is_regi...",USD,$,True,...,apple-watch-development-course,https://www.kickstarter.com/discover/categorie...,False,False,failed,1428511019,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",549.00000,domestic


In [37]:
# Prints out the columns with NaNs
get_nan_cols(df)

Number of NaNs per column:
blurb:8
friends:208922
is_backing:208922
is_starred:208922
location:226
permissions:208922
usd_type:480


In [38]:
df[['pledged', 'usd_pledged', 'converted_pledged_amount']]

,pledged,usd_pledged,converted_pledged_amount
0,802.0,802.000000,802
1,2259.0,2259.000000,2259
2,29638.0,29638.000000,29638
3,43180.0,49075.152520,49158
4,549.0,549.000000,549
...,...,...,...
960,10320.0,10320.000000,10320
961,435.0,316.495068,305
962,0.0,0.000000,0
963,1400.0,1400.000000,1400


We can get rid of some columns, which probably will have no impact on the model later on or are actual data leakage, as they contain future information. 
Columns for the `pledged money`, the `backers_count`, `spotlight` and `staff_picked` are future knowledge so we will drop these.

Especially `friends`, `is_backing`, `is_starred`, `permission` are almost completely empty (over 50% of data was missing). So no information loss here.

Moreover the following columns don't seem to contain any valuable information for the predictions as well: `currency_symbol`, `currency_trailing_code`, `fx_rate`, `id`, `photo`,`profile`,`slug`,`source_url`,`static_usd_rate`,`urls`.

The 226 and 480 missing observations for `location` and `usd_type` are so small in number that we can get rid of these rows as well. 

We also rename the `currency` into `original_currency` and `name` into `project_name` as this gives more information about its content. 

In [39]:
# Convert the column names to a list
columns = df.columns.to_list()
# Drop all columns with more than 50% of the observations missing
df = df[[column for column in df if df[column].count() / len(df) >= 0.5]]
# Print the dropped columns
print("Dropped columnes:", end= " ")
dropped = [print(col, end=" ") for col in columns if col not in df.columns]

Dropped columnes: friends is_backing is_starred permissions 

In [40]:
# Drop the listed columns
df.drop([
    'backers_count',
    'converted_pledged_amount',
    'currency_symbol', 
    'currency_trailing_code', 
    'fx_rate',
    'id',
    'pledged',
    'photo',
    'profile',
    'slug',
    'source_url', 
    'spotlight',
    'staff_pick',
    'static_usd_rate',
    'urls',
    'usd_pledged'
    ], axis=1, inplace=True)
# Drops the last few NaN values
df.dropna(axis=0, inplace=True)
# Rename the currency column
df.rename(columns={
    'currency':'original_currency',
    'name':'project_name'}, inplace=True)

In [41]:
# Prints out the columns with NaNs
get_nan_cols(df)

Number of NaNs per column:


Now we have no NaNs left in our data set, which is another step towards a cleaned data set.

The columns of `created_at`, `launched_at`, `state_changed_at`, `deadline` are actually dates but expressed as epoch time. We will convert these into datetime objects and thereby into actual dates.

With these we can calculate the time periods from the beginning of the project until the success and also to the deadline of the project.

Moreover, from these dates we can then get the year, month and day of the three columns.

In [42]:
# Convert the time columns to datetime types
df = convert_to_datetime(df, ['created_at', 'state_changed_at', 'deadline', 'launched_at'])
# Calculate the time periods
df = calculate_time_periods(df)
# Get the years, months and days as separate columns
df = get_year_month_day(df, ['created_at', 'state_changed_at', 'deadline', 'launched_at'])

In [43]:
df.head()

,blurb,category,country,creator,original_currency,current_currency,disable_communication,goal,is_starrable,location,...,state_changed_at_day,state_changed_at_weekday,deadline_year,deadline_month,deadline_day,deadline_weekday,launched_at_year,launched_at_month,launched_at_day,launched_at_weekday
0,2006 was almost 7 years ago.... Can you believ...,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po...",US,"{""id"":1495925645,""name"":""Daniel"",""is_registere...",USD,USD,False,200.0,False,"{""id"":2379574,""name"":""Chicago"",""slug"":""chicago...",...,8,5,2014,2,8,5,2013,12,25,2
1,An adorable fantasy enamel pin series of princ...,"{""id"":54,""name"":""Mixed Media"",""slug"":""art/mixe...",US,"{""id"":1175589980,""name"":""Katherine"",""slug"":""fr...",USD,USD,False,400.0,False,"{""id"":2486340,""name"":""Sacramento"",""slug"":""sacr...",...,5,1,2019,3,5,1,2019,2,13,2
2,Helping a community come together to set the s...,"{""id"":280,""name"":""Photobooks"",""slug"":""photogra...",US,"{""id"":1196856269,""name"":""MelissaThomas"",""is_re...",USD,USD,False,27224.0,False,"{""id"":2383660,""name"":""Columbus"",""slug"":""columb...",...,1,3,2016,12,1,3,2016,11,1,1
3,Every revolution starts from the bottom and we...,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",IT,"{""id"":1569700626,""name"":""WAO"",""slug"":""wearewao...",EUR,USD,False,40000.0,False,"{""id"":725746,""name"":""Venice"",""slug"":""venice-ve...",...,8,5,2018,12,8,5,2018,10,27,5
4,Learn to build 10+ Applications in this comple...,"{""id"":51,""name"":""Software"",""slug"":""technology/...",US,"{""id"":1870845385,""name"":""Kalpit Jain"",""is_regi...",USD,USD,False,1000.0,False,"{""id"":2479651,""name"":""Redmond"",""slug"":""redmond...",...,8,2,2015,4,8,2,2015,3,9,0


In [44]:
print("Percentage of the data where the project duration is the same as the time periode to reach the goal: ", end='')
print((df[df['days_launched_till_changed']==df['days_total']].shape[0] / len(df))*100)

Percentage of the data where the project duration is the same as the time periode to reach the goal: 8.211675331402152


Only 8% of the projects were initiated immediately after their creation and changed their status at the same time as the deadline. 

Let's look further into the data types of the other columns.

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208508 entries, 0 to 964
Data columns (total 32 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   blurb                       208508 non-null  object 
 1   category                    208508 non-null  object 
 2   country                     208508 non-null  object 
 3   creator                     208508 non-null  object 
 4   original_currency           208508 non-null  object 
 5   current_currency            208508 non-null  object 
 6   disable_communication       208508 non-null  bool   
 7   goal                        208508 non-null  float64
 8   is_starrable                208508 non-null  bool   
 9   location                    208508 non-null  object 
 10  project_name                208508 non-null  object 
 11  state                       208508 non-null  object 
 12  usd_type                    208508 non-null  object 
 13  days_launched_til

In [46]:
for col in ['category', 'creator', 'location']:
    print(df[col].iloc[0], '\n')

{"id":43,"name":"Rock","slug":"music/rock","position":17,"parent_id":14,"color":10878931,"urls":{"web":{"discover":"http://www.kickstarter.com/discover/categories/music/rock"}}} 

{"id":1495925645,"name":"Daniel","is_registered":null,"chosen_currency":null,"avatar":{"thumb":"https://ksr-ugc.imgix.net/assets/006/041/047/c44d1a95c2139ae46af635c7c6e7ea76_original.jpg?ixlib=rb-1.1.0&w=40&h=40&fit=crop&v=1461362658&auto=format&frame=1&q=92&s=3d655afafac9dbb59c1e675adfa87082","small":"https://ksr-ugc.imgix.net/assets/006/041/047/c44d1a95c2139ae46af635c7c6e7ea76_original.jpg?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1461362658&auto=format&frame=1&q=92&s=3973d24f5c3db1ed1d5c84cec8af1d6d","medium":"https://ksr-ugc.imgix.net/assets/006/041/047/c44d1a95c2139ae46af635c7c6e7ea76_original.jpg?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1461362658&auto=format&frame=1&q=92&s=3973d24f5c3db1ed1d5c84cec8af1d6d"},"urls":{"web":{"user":"https://www.kickstarter.com/profile/1495925645"},"api":{"user":"https://api.kick

We find very strange formatted object type data in `category`, `creator` and `location` column. The next step will be to entangle these strings and get the most important information out of them. For the location we could extract the city name of the project creator. For the creator we could get their names for now. And for the category the id and the name could be interesting.

In [47]:
# Entangles the category, creator and location column
df = entangle_column(df, ['category', 'creator', 'location'])
df.rename(columns={'category_name':'category'}, inplace=True)

In [48]:
df.head()

,blurb,country,original_currency,current_currency,disable_communication,goal,is_starrable,project_name,state,usd_type,...,deadline_weekday,launched_at_year,launched_at_month,launched_at_day,launched_at_weekday,category,category_id,category_sub,creator_name,location_name
0,2006 was almost 7 years ago.... Can you believ...,US,USD,USD,False,200.0,False,New Final Round Album,successful,international,...,5,2013,12,25,2,music,43,rock,Daniel,Chicago
1,An adorable fantasy enamel pin series of princ...,US,USD,USD,False,400.0,False,Princess Pals Enamel Pin Series,successful,international,...,1,2019,2,13,2,art,54,mixed media,Katherine,Sacramento
2,Helping a community come together to set the s...,US,USD,USD,False,27224.0,False,Their Life Through Their Lens-the Amish and Me...,successful,international,...,3,2016,11,1,1,photography,280,photobooks,MelissaThomas,Columbus
3,Every revolution starts from the bottom and we...,IT,EUR,USD,False,40000.0,False,WAO: THE ECO EFFECT SHOES,successful,international,...,5,2018,10,27,5,fashion,266,footwear,WAO,Venice
4,Learn to build 10+ Applications in this comple...,US,USD,USD,False,1000.0,False,Apple Watch Development Course,failed,domestic,...,2,2015,3,9,0,technology,51,software,Kalpit Jain,Redmond


Now we should take a look at the number of unique observations for each column.

In [49]:
# Show the amount of unique items per column
df.nunique()

blurb                         180441
country                           22
original_currency                 14
current_currency                   1
disable_communication              2
goal                            5103
is_starrable                       2
project_name                  181417
state                              5
usd_type                           2
days_launched_till_changed        94
days_prelaunch                  1467
days_total                      1497
created_at_year                   11
created_at_month                  12
created_at_day                    31
created_at_weekday                 7
state_changed_at_year             11
state_changed_at_month            12
state_changed_at_day              31
state_changed_at_weekday           7
deadline_year                     11
deadline_month                    12
deadline_day                      31
deadline_weekday                   7
launched_at_year                  11
launched_at_month                 12
l

If we look at the number of unique entries for each column, we can see that the current currency has only one entry, USD. Since this does not add any information to the model, we can also drop this column. 

In [50]:
# Drop the current_currency column
df.drop('current_currency', axis=1, inplace=True)

In [51]:
df.head()

,blurb,country,original_currency,disable_communication,goal,is_starrable,project_name,state,usd_type,days_launched_till_changed,...,deadline_weekday,launched_at_year,launched_at_month,launched_at_day,launched_at_weekday,category,category_id,category_sub,creator_name,location_name
0,2006 was almost 7 years ago.... Can you believ...,US,USD,False,200.0,False,New Final Round Album,successful,international,45,...,5,2013,12,25,2,music,43,rock,Daniel,Chicago
1,An adorable fantasy enamel pin series of princ...,US,USD,False,400.0,False,Princess Pals Enamel Pin Series,successful,international,20,...,1,2019,2,13,2,art,54,mixed media,Katherine,Sacramento
2,Helping a community come together to set the s...,US,USD,False,27224.0,False,Their Life Through Their Lens-the Amish and Me...,successful,international,30,...,3,2016,11,1,1,photography,280,photobooks,MelissaThomas,Columbus
3,Every revolution starts from the bottom and we...,IT,EUR,False,40000.0,False,WAO: THE ECO EFFECT SHOES,successful,international,42,...,5,2018,10,27,5,fashion,266,footwear,WAO,Venice
4,Learn to build 10+ Applications in this comple...,US,USD,False,1000.0,False,Apple Watch Development Course,failed,domestic,30,...,2,2015,3,9,0,technology,51,software,Kalpit Jain,Redmond


Other columns that give us probably no further information are the `category id` and also the `location_name`, since the `country` is probably more important.

In [52]:
df.drop(['category_id', 'location_name'], axis=1, inplace=True)

The only valuable information we can get out of the names without NLP is their length. So we calculate the length of the name columns and drop the name columns.

In [56]:
get_char_len(df, columns=['project_name', 'creator_name', 'blurb'])

In [57]:
df.head()

,country,original_currency,disable_communication,goal,is_starrable,state,usd_type,days_launched_till_changed,days_prelaunch,days_total,...,deadline_weekday,launched_at_year,launched_at_month,launched_at_day,launched_at_weekday,category,category_sub,project_name_len,creator_name_len,blurb_len
0,US,USD,False,200.0,False,successful,international,45,4,49,...,5,2013,12,25,2,music,rock,21,6,134
1,US,USD,False,400.0,False,successful,international,20,5,25,...,1,2019,2,13,2,art,mixed media,31,9,55
2,US,USD,False,27224.0,False,successful,international,30,9,39,...,3,2016,11,1,1,photography,photobooks,60,13,135
3,IT,EUR,False,40000.0,False,successful,international,42,3,45,...,5,2018,10,27,5,fashion,footwear,25,3,75
4,US,USD,False,1000.0,False,failed,domestic,30,2,32,...,2,2015,3,9,0,technology,software,30,11,133


Now let's look at the basic descriptive statistics of the numeric columns.

In [58]:
# Create a list of the real numeric columns
numeric_cols = [
    'state',
    'goal', 
    'days_prelaunch',
    'days_launched_till_changed',
    'days_total',
    'project_name_len',
    'creator_name_len',
    'blurb_len']
# Get the basic descriptive statistics for the numeric columns
df[numeric_cols].describe()

,goal,days_prelaunch,days_launched_till_changed,days_total,project_name_len,creator_name_len,blurb_len
count,2.085080e+05,208508.000000,208508.000000,208508.000000,208508.000000,208508.000000,208508.000000
mean,4.925714e+04,47.598529,30.548574,80.341958,35.457062,14.320726,111.758335
std,1.181417e+06,131.112428,13.314202,131.919612,15.705974,5.968540,27.193935
min,1.000000e-02,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,1.500000e+03,3.000000,28.000000,33.000000,22.000000,11.000000,100.000000
50%,5.000000e+03,11.000000,30.000000,47.000000,35.000000,13.000000,123.000000
75%,1.500000e+04,37.000000,33.000000,73.000000,49.000000,16.000000,132.000000
max,1.000000e+08,3304.000000,93.000000,3334.000000,95.000000,64.000000,196.000000


The next step is to one-hot-encode all the categorical features.

In [60]:
# One-hot-encode the categorical features.
df = one_hot_encode(df, numeric_cols)

In [61]:
df.head()

,goal,state,days_launched_till_changed,days_prelaunch,days_total,project_name_len,creator_name_len,blurb_len,country_AT,country_AU,...,category_sub_wearables,category_sub_weaving,category_sub_web,category_sub_webcomics,category_sub_webseries,category_sub_woodworking,category_sub_workshops,category_sub_world music,category_sub_young adult,category_sub_zines
0,200.0,successful,45,4,49,21,6,134,0,0,...,0,0,0,0,0,0,0,0,0,0
1,400.0,successful,20,5,25,31,9,55,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27224.0,successful,30,9,39,60,13,135,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40000.0,successful,42,3,45,25,3,75,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000.0,failed,30,2,32,30,11,133,0,0,...,0,0,0,0,0,0,0,0,0,0


For our target, we also have to do some cleaning. Live projects give us no information since their outcome is unclear for now. So we will focus on the other projects. The data loss is also not that large. We will also group suspended, aborted, and failed projects together so that our target consists only of successful and failed projects. The reason for this is that we are most interested in the successful projects and the reasons why a project failed, no matter in which way (suspended, aborted, failed) it failed.

In [62]:
df['state'].unique()

array(['successful', 'failed', 'live', 'canceled', 'suspended'],
      dtype=object)

In [26]:
df.shape

(208516, 473)

In [28]:
# Get rid of the live state entries
df= df[df['state'] != 'live']
df.shape

(201647, 473)